<a href="https://colab.research.google.com/github/rainbowcity05/tibame0410/blob/main/%E3%80%8Cpchome_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import urllib.request as req
import bs4 as bs
import os
import pandas as pd

#建立資料夾
dn = "pchome/"
if not os.path.exists(dn):
    os.mkdir(dn)

#準備table
table = []

url = "https://24h.pchome.com.tw/search/?q=%E5%90%B9%E9%A2%A8%E6%A9%9F"
resp = req.urlopen(url)
html = bs.BeautifulSoup(resp.read())
links = html.find_all("a", {"class":"c-prodInfoV2__link"})
for link in links:
    link_href = "https://24h.pchome.com.tw" + link["href"]
    p_resp = req.urlopen(link_href)
    p_html = bs.BeautifulSoup(p_resp.read())

    # 產品品牌
    p_brand = p_html.find("span", {"class":"o-prodMainName__colorSecondary"}).text

    # 產品名稱
    p_name = p_html.find("h1", {"class":"o-prodMainName__grayDarkest"}).text
    p_name_text = p_name.strip()

    #評分
    p_rating_all = p_html.find("div", {"class":"c-ratingIcon__flex"})
    if not p_rating_all == None:
        p_rating = p_rating_all.find("div", {"class":"c-ratingIcon__textNumber"})
        p_rating_count = p_rating_all.find("a")
        p_rating_text = p_rating.text.strip()
        p_rating_count_text = p_rating_count.text.strip().replace("則評價", "")
    else:
        p_rating_text = "-"
        p_rating_count_text = "-"
    #介紹
    p_intro_list = p_html.find_all("li", {"class":"c-blockCombine__item--prodSlogan"})
    p_intro_text_list = []
    for p_intro in p_intro_list:
        p_intro_text = p_intro.text.strip()
        p_intro_text_list.append(p_intro_text)
    p_intro_text_join = ""
    for p_intro_text in p_intro_text_list:
        p_intro_text_join = p_intro_text_join + p_intro_text + "\n"

    #價格
    p_discount_price = p_html.find("div", {"class":"o-prodPrice__price"}).text
    p_original_price = p_html.find("div", {"class":"o-prodPrice__originalPrice"})
    if not p_original_price == None:
      p_original_price = p_original_price.text
    else:
      p_original_price = "-"

    #抓圖片url
    p_img = p_html.find("div", {"class":"c-radiusPhotoImage__img"}).find("img")
    p_img_source = p_img["src"]
    if not "https:" in p_img_source:
        p_img_source = "https:" + p_img_source
    #圖片存檔
    fn = link_href.split("/")[-1]
    fn = dn + fn +".jpg"
    req.urlretrieve(p_img_source,fn)


    #滿額贈
    p_labels = p_html.find_all("div", {"class":"c-label__rectangle"})
    p_full_prize = "N"
    for p_label in p_labels:
        if "滿額贈" in p_label.text:
            p_full_prize = "Y"


    # print(p_brand)
    # print(p_name)
    # print(p_rating_text)
    # print("幾則評價:",p_rating_count_text)
    # print(p_intro_text_join)
    # print("折扣價:",p_discount_price)
    # print("原價:",p_original_price)
    # print(p_full_prize)
    # print(p_img_source)
    # print("*"*30)


    #建立raw data
    data = {
      "品牌":p_brand,
      "名稱":p_name,
      "評分":p_rating_text,
      "幾則":p_rating_count_text,
      "介紹":p_intro_text_join,
      "現價":p_discount_price,
      "原價":p_original_price,
      "滿額贈":p_full_prize,
      "圖片":p_img_source,
      "圖片儲存":fn
    }
    table.append(data)

df = pd.DataFrame(table)
df.to_csv("pchome.csv", encoding="utf-8")
df




,品牌,名稱,評分,幾則,介紹,現價,原價,滿額贈,圖片,圖片儲存
0,sOlac,國際電壓專業負離子吹風機 T型吹風機 BLDC輕巧輕量吹風機（SD-1000Pro 白金色）,-,-,國際電壓 出國必備\n輕巧美型 簡約大方\n千萬負離子 可反向清潔\n,"$4,680","$5,280",Y,https://img.pchome.com.tw/cs/items/DMAH47A900I...,pchome/DMAH47-A900IH7HT.jpg
1,AVEDA,蘊活煥欣洗髮精 1000ml,4.5,2,去除頭皮老廢角質與殘留物堆積\n天然起泡成分，溫和清潔\n細軟髮或落髮問題特別推薦\n,"$5,400","$6,000",Y,https://img.pchome.com.tw/cs/items/DDDU04A900H...,pchome/DDDU04-A900HHN7T.jpg
2,AVEDA,蘊活迷迭養護組,-,-,蘊活系列呵護髮肌，秀髮豐盈新生\n迷迭薄荷清新經典花香\n細軟髮或落髮問題特別推薦\n,"$2,551","$2,551",Y,https://img.pchome.com.tw/cs/items/DDBXFA1900I...,pchome/DDBXFA-1900IFCIR.jpg
3,acerpure 宏碁,Acerpure beauty 雙效負離子吹風機 HD364-10W,-,-,"ProDry多風道氣流雙效吹嘴\n2,200萬高濃度負離子\n便利收納 沙龍級享受隨行\n","$1,290",-,N,https://img.pchome.com.tw/cs/items/DMAHE21900I...,pchome/DMAHE2-1900IAF6I.jpg
4,AVEDA,蘊活淨瑕頭皮調理液 100ml (鎮靜紓緩頭皮並祛除頭氣),5.0,8,48小時清爽控油，呵護髮肌\n免沖洗直接噴於頭皮並搭配按摩\n可於日間隨時使用\n,"$1,900","$2,100",Y,https://img.pchome.com.tw/cs/items/DDBH8BA900G...,pchome/DDBH8B-A900GK0TK.jpg
5,Xiaomi 小米,負離子高速吹風機,4.9,80,高濃度負離子 | 中和靜電，減少毛躁\n400g輕巧機身 | 長時間使用也不疲累\n8種出風...,"$1,295","$1,295",N,https://img.pchome.com.tw/cs/items/DMAHBYA900I...,pchome/DMAHBY-A900I0K1F.jpg
6,KINYO,全球電壓無刷吹風機 KH-9660,5.0,4,100-240V全球電壓自動切換，隨插即吹，使用地區不受限\n90度折疊機身設計，節省空間，...,"$2,780","$4,480",N,https://cs-a.ecimg.tw/items/DMAHCWA900IGQBP/yt...,pchome/DMAHCW-A900IGQBP.jpg
7,Panasonic 國際牌,高速馬達吹風機 EH-NE6M-C 焦糖奶茶色,4.9,53,"直線風梳技術 水平出風口吹出直線風流,使頭髮 不易打結,更滑順。\nBLDC高速馬達 提供強...","$2,799","$2,990",Y,https://img.pchome.com.tw/cs/items/DMAH4CA900I...,pchome/DMAH4C-A900I7YVV.jpg
8,Panasonic 國際牌,奈米水離子吹風機(EH-NA27-W),4.9,301,奈米水離子潤澤滲透護髮科技\n加頭髮的保濕度\n頭皮到髮尾完整保養\n,"$1,688","$2,190",Y,https://img.pchome.com.tw/cs/items/DMAH02A900A...,pchome/DMAH02-A900AN6LF.jpg
9,Panasonic 國際牌,高速馬達吹風機 EH-NE6M-A 寶寶粉藍色,5.0,27,"直線風梳技術 水平出風口吹出直線風流,使頭髮 不易打結,更滑順。\nBLDC高速馬達 提供強...","$2,799","$2,990",Y,https://img.pchome.com.tw/cs/items/DMAH4CA900I...,pchome/DMAH4C-A900I7YVH.jpg


In [11]:
import requests
import json
url = "https://www.thsrc.com.tw/TimeTable/Search"
#將網頁抓的payload存成長字串
payload_s = """SearchType: S
Lang: TW
StartStation: TaiPei
EndStation: ZuoYing
OutWardSearchDate: 2025/04/28
OutWardSearchTime: 15:30
ReturnSearchDate: 2025/04/28
ReturnSearchTime: 15:30
DiscountType: """

payload = {} #建立一個空字典
for spl in payload_s.split("\n"): #將原本的長字串按行分開
    k, v = spl.split(": ") #用分號將每行分成key值和value值
    k, v = k.strip(), v.strip() #去除多餘的空白
    payload[k] = v #將每個key值對應的value加進字典中，做出字典型式的payload

resp = requests.post(url, payload) #發出帶payload的post需求，並且將回應存成resp
resp_json = json.loads(resp.text) #將回應的json字串轉成python的字典格式

data = resp_json["data"] #從字典中取出data的value值
trains = data["DepartureTable"]["TrainItem"] #從data裡的DepartureTable再取出TrainItem，存成trains
df = pd.json_normalize(trains) #將每個train item轉換成表格
print(type(df))
df.to_csv("trains.csv",index = False, encoding = "utf-8")

<class 'pandas.core.frame.DataFrame'>
